## RAG

In [9]:
model_id = "../Llama-2-7b-chat-hf"
input_path = "vector_db"

In [10]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain import HuggingFacePipeline

In [12]:
# Load db
vectordb = Chroma(persist_directory=input_path, embedding_function=HuggingFaceEmbeddings())
print(vectordb._collection.count())

154


In [13]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# # Load model directly
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id)
# model.to(device)

# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)

# llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [14]:
import os
import openai
from langchain.llms import OpenAI
from dotenv import load_dotenv
import torch


load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)

In [15]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Utiliza los siguientes elementos de contexto para responder a la pregunta del final. Genera una única respuesta. Si no sabes la respuesta, di simplemente que no la sabes, no intentes inventarte una respuesta. Utiliza tres frases como máximo. La respuesta debe ser lo más concisa posible.
{context}
Pregunta: {question}
Respuesta:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [16]:
def imprimir_respuesta(result):
    answer = result["result"]
    print(f"PREGUNTA: {result['query']}")
    print(f"RESPUESTA: {answer}")
    print("FUENTES:")
    for el in result["source_documents"]:
        # print("{" + f"'source': '{el.metadata['source']}', 'page': '{el.metadata['page']}', 'capitulo': '{el.metadata['capitulo']}', 'articulo': '{el.metadata['articulo']}" + "}")
        print(f"Fuente: {el.metadata['source']}")
        print(el.page_content)

In [17]:
question = "¿Cómo se puede disolver el grupo? Dime los pasos y condiciones"
result = qa_chain.invoke({"query": question})

imprimir_respuesta(result)

PREGUNTA: ¿Cómo se puede disolver el grupo? Dime los pasos y condiciones
RESPUESTA:  El grupo puede ser disuelto por decisión de la Asamblea General Extraordinaria, convocada específicamente para este fin y con una mayoría de dos tercios de los votos de los socios presentes. Una vez tomada la decisión, se procederá a la liquidación del patrimonio del grupo y a la cancelación de su inscripción en el registro de asociaciones.
FUENTES:
Fuente: 2. Programa Educativo de Grupo 2022 - 2025
```
El Grupo Scout Annapurna, en adelante “el Grupo” se define como un conjunto de personas
que trabaja, aprende y forma en el escultismo. Es una entidad autogestionada, no lucrativa
y de carácter social, apartidista, aconfesional y dedicada a la labor educativa de jóvenes
comprendidos entre 6 y 21 años.
```
```
El Grupo está formado por diferentes agentes, siendo estos los educandos, sus familias, el
grupo de colaboradores, el equipo de responsables, el grupo de antiguos responsables, el
barrio y la asocia

In [18]:
question = "¿Quien es el Kraal?"
result = qa_chain.invoke({"query": question})

imprimir_respuesta(result)

PREGUNTA: ¿Quien es el Kraal?
RESPUESTA:  El Kraal es el conjunto de responsables del Grupo encargados de liderar y gestionar las actividades educativas y formativas de la organización, y su elección y admisión se realiza mediante consenso del mismo.
FUENTES:
Fuente: Estatutos (2022)
La elección  y admisión  de los nuevos  miembros  del equipo  de Kraal  se realizará   mediante el consenso del mismo.
Fuente: 2. Programa Educativo de Grupo 2022 - 2025
Las familias, antiguas personas del equipo de Kraal y conocidos del Kraal actual tienen la
opción de ayudar con las diferentes tareas de los cargos durante el trimestre y los
campamentos.
Fuente: Reglamento de Regimen Interno (2023)
```
El Equipo de Kraal es el conjunto de responsables del Grupo encargados del
funcionamiento de esta y del cumplimiento de los fines de grupo mediante la
preparación, desarrollo y supervisión de las actividades. El Equipo de Kraal
actuará en consecuencia a las decisiones que tome la Asamblea General.
```
Fuent

In [19]:
question = "¿En que equipos se divide el Kraal? ¿Que cargo está en cada equipo?"
result = qa_chain.invoke({"query": question})

imprimir_respuesta(result)

PREGUNTA: ¿En que equipos se divide el Kraal? ¿Que cargo está en cada equipo?
RESPUESTA:  No se especifica en el contexto la división del Kraal en equipos ni los cargos que corresponden a cada uno.
FUENTES:
Fuente: Estatutos (2022)
La elección  y admisión  de los nuevos  miembros  del equipo  de Kraal  se realizará   mediante el consenso del mismo.
Fuente: Reglamento de Regimen Interno (2023)
```
El Equipo de Kraal es el conjunto de responsables del Grupo encargados del
funcionamiento de esta y del cumplimiento de los fines de grupo mediante la
preparación, desarrollo y supervisión de las actividades. El Equipo de Kraal
actuará en consecuencia a las decisiones que tome la Asamblea General.
```
Fuente: Estatutos (2022)
El equipo de Kraal  es el conjunto de responsables de la Asociación  encargados  del funcionamiento de la misma y del cumplimiento de los fines de grupo mediante la  preparación, desarrollo y supervisión de las actividades. Está formado por asociados  mayores d e edad cuy

In [20]:
question = "¿Que son las comisiones de grupo?"
result = qa_chain.invoke({"query": question})

imprimir_respuesta(result)

PREGUNTA: ¿Que son las comisiones de grupo?
RESPUESTA:  Las comisiones de grupo son grupos de trabajo formados por personas socias educandas o sus representantes legales, personas socias educadoras y personas socias de honor del Grupo, que están interesadas en participar en las diferentes comisiones y consejos juveniles. Estos equipos funcionan como órganos de consulta y propuesta, pero las decisiones finales las toma el Equipo de Kraal.
FUENTES:
Fuente: Reglamento de Regimen Interno (2023)
```
Son aquellos grupos de trabajo compuestos por cualquier persona asociada del
Grupo o sus representantes legales. Estos equipos funcionan como órganos
de consulta y propuesta, acerca del funcionamiento del grupo o sobre las
actividades que se realizan, pero las decisiones finales las toma el Equipo de
Kraal.
```
```
Las comisiones actuales son las siguientes:
```
```
a. Las Comisiones Juveniles de Grupo.
b. Los Consejos de Jóvenes.
c. Los Consejos de Familias.
d. El Consejo de Promesa.
```
Fuente

In [21]:
question = "¿Para que sirve el consejo de familias?"
result = qa_chain.invoke({"query": question})

imprimir_respuesta(result)

PREGUNTA: ¿Para que sirve el consejo de familias?
RESPUESTA:  El consejo de familias sirve como un espacio de diálogo y consulta entre las familias y el Kraal, donde se pueden abordar temas relacionados con el grupo y brindar apoyo y consejos en caso de problemas o preocupaciones.
FUENTES:
Fuente: 2. Programa Educativo de Grupo 2022 - 2025
```
Se realizan trimestralmente en las reuniones de los campamentos, Navidad, Semana
Santa y Verano respectivamente; así como en la reunión de Inicio de Ronda y en el día de
Familias del Campamento de Verano. Consistirá en un diálogo abierto donde ambas
partes debatirán temas concernientes al grupo, por ejemplo, las familias podrán exponer
sus preocupaciones o el Kraal podrá comunicar problemas que puedan llegar a tener y
pedir consejos o ayuda a estas.
```
Fuente: 2. Programa Educativo de Grupo 2022 - 2025
```
Las familias dentro del grupo mantienen una comunicación constante con el Kraal por medio
de distintos grupos de WhatsApp, correos informativ

In [22]:
question = "¿Cuales son los requisitos para ser elegible como miembro de la junta directiva? ¿Cuales son los cargos? ¿Cuanto dura el mandato?"
result = qa_chain.invoke({"query": question})

imprimir_respuesta(result)

# 62.1.- Requisitos para ser elegible:
# a. Ser parte del equipo de Kraal.
# b. Ser mayor de edad.
# c. Ser una persona asociada del grupo.
# d. No haber sido elegido previamente el mismo cargo durante al menos una candidatura.

PREGUNTA: ¿Cuales son los requisitos para ser elegible como miembro de la junta directiva? ¿Cuales son los cargos? ¿Cuanto dura el mandato?
RESPUESTA:  Los requisitos para ser elegible como miembro de la junta directiva son estar en pleno uso de los derechos civiles y no estar incursos en motivos de incompatibilidad legalmente establecidos. Los cargos son la Presidencia, la Secretaría, Tesorería y 2 Vocalías designadas en la Asamblea General. El mandato tiene una duración de dos años.
FUENTES:
Fuente: 2. Programa Educativo de Grupo 2022 - 2025
```
La Junta Directiva forma parte del equipo de Kraal. Esta formada por la Presidencia, la
Secretaría, Tesorería y 2 Vocalías designadas en la Asamblea General, en pleno uso de sus
derechos civiles que no estén incursos en motivos de incompatibilidad legalmente
establecidos. Su mandato tendrá una duración de dos años. La Presidencia y la Secretaría
de la Junta Directiva lo serán, asimismo, de la Asociación y de la Asamblea General. Todos
los mie

In [23]:
question = "¿Cuales sons los artículos de la ley scout?"
result = qa_chain.invoke({"query": question})

imprimir_respuesta(result)

PREGUNTA: ¿Cuales sons los artículos de la ley scout?
RESPUESTA:  Los artículos de la ley scout son diez y se refieren a ser digno de confianza, leal, servicial, amigo de todos y hermano de cualquier otro scout, cortés y educado, amante y protector de la naturaleza, responsable y no hacer nada a medias, animoso ante peligros y dificultades, trabajador y austero, y limpio y sano en pensamientos, palabras y acciones.
FUENTES:
Fuente: Reglamento de Regimen Interno (2023)
```
La Ley Scout es un instrumento educativo. A través de ésta, se exponen de forma
positiva, con un lenguaje sencillo, los principios y valores que nos guían y conforman
el estilo de vida a seguir por quienes asumimos la Promesa. Cada sección, sin variar
el significado, hace sus propias adaptaciones al nivel y situaciones de sus jóvenes,
traduciendo a la vida diaria sus necesidades. Supone un reto para la imaginación de
toda persona que vive de verdad su compromiso: “La Ley Scout es la base sobre la
que descansa toda la 